1. Попробуйте изменить параметры нейронной сети работающей с датасетом imdb так, чтобы улучшить ее точность. Приложите анализ.

In [37]:
%%time
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 150
batch_size = 100 # увеличьте значение для ускорения обучения

print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

print('Pad последовательности (примеров в x единицу времени)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Построение модели...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров 
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1, # увеличьте при необходимости
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)


Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности
Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 150)
x_test shape: (25000, 150)
Построение модели...
Процесс обучения...
250/250 [==============================] - 23s 91ms/step - loss: 0.3138 - accuracy: 0.8672
Результат при тестировании: 0.3137646019458771
Тестовая точность: 0.8672000169754028
Wall time: 3min 43s


Результат модели обычной:
- Тестовая точность: 0.8418400287628174

Заменим сигмойды на relu:
- Тестовая точность: 0.8352400064468384
- Особой разницы между сигмойдой и relu в данном примере нет

Увеличим кол-во нейронов со 128 до 256:
- Тестовая точность: 0.8409600257873535
- Увеличилось существенно время обучения модели, но качество не изменилось

Увеличим размер анализируемого отзыва с 80 до 150 слов:
- Тестовая точность: 0.8661199808120728
- Точность существенно выросла благодаря большему кол-ву входных данных для анализа

Сделаем 5 эпох и размер отзывов - 150 слов:
- Тестовая точность: 0.8489599823951721
- Модель ухудшилась и время обучения существенно выросло. Увеличение эпох с 1 до 5 видимо приводит к переобучению при низком кол-ве входных данных (150 слов на отзыв)

Увеличим batch_size с 50 до 100 и размер отзывов - 150 слов:
- Тестовая точность: 0.8672000169754028
- Точность не поменялась, зато скорость обучения заметно выросла

__Итого: значительный вклад в увеличение точности модели вносит размер входных данных (т.е. кол-во слов в отзыве) - чем их больше, тем выше будет точность до некоторых пределов.__


2. Попробуйте изменить параметры нейронной сети генерирующий текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший получившейся у вас текст и опишите, что вы предприняли, чтобы его получить. Можно использовать текст другого прозведения.

In [38]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential

In [39]:
# построчное чтение из примера с текстом 
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)

В тексте в начале много текста, не относящегося к произведению. Удалим его

Также удалим символы *, слово chapter и латинские цифры после chapter, и текст после слов THE END

In [40]:
text

'project gutenbergs alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  you may copy it, give it away or re-use it under the terms of the project gutenberg license included with this ebook or online at www.gutenberg.org title: alices adventures in wonderland author: lewis carroll posting date: june 25, 2008 [ebook #11] release date: march, 1994 last updated: october 6, 2016 language: english character set encoding: utf-8 *** start of this project gutenberg ebook alices adventures in wonderland *** alices adventures in wonderland lewis carroll the millennium fulcrum edition 3.0 chapter i. down the rabbit-hole alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, and what is the use of a book, thought alice without pictures or c

In [85]:
text2 = text.split('chapter ')
text2[12] = text2[12].split('. the end')[0]

In [86]:
for i in range(1, 13):
    text2[i] = text2[i][3:]
    text2[i] = text2[i].replace('*', '').strip()
for i in [3,7,8,12]:
    text2[i] = text2[i][2:]

In [92]:
text3 = ''.join(text2[1:])

In [93]:
text3

'down the rabbit-hole alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, and what is the use of a book, thought alice without pictures or conversations? so she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a white rabbit with pink eyes ran close by her. there was nothing so very remarkable in that; nor did alice think it so very much out of the way to hear the rabbit say to itself, oh dear! oh dear! i shall be late! (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the rabbit actually took a watch out of its waistcoat-pocket, and look

In [115]:
# построчное чтение из примера с текстом 
with open("Чехов Антон. Драма на охоте.txt", 'r', encoding = 'cp1251') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower()
        if len(line) == 0:
            continue
        lines.append(line)
text3 = " ".join(lines)
text3 = text3.replace('\xa0', ' ')

In [116]:
chars = set([c for c in text3])
nb_chars = len(chars)

In [117]:
text3

'в один из апрельских полудней 1880 года в мой кабинет вошел сторож андрей и таинственно доложил мне, что в редакцию явился какой-то господин и убедительно просит свидания с редактором. — должно быть, чиновник-с, — добавил андрей, — с кокардой… — попроси его прийти в другое время, — сказал я. — сегодня я занят. скажи, что редактор принимает только по субботам. — он и третьего дня приходил, вас спрашивал. говорит, что дело большое. просит и чуть не плачет. в субботу, говорит, ему несвободно… прикажете принять? я вздохнул, положил перо и принялся ждать господина с кокардой. начинающие писатели и вообще люди, не посвященные в редакционные тайны, приходящие при слове «редакция» в священный трепет, заставляют ждать себя немалое время. они, после редакторского «проси», долго кашляют, долго сморкаются, медленно отворяют дверь, еще медленнее входят и этим отнимают немало времени. господин же с кокардой не заставил ждать себя. не успела за андреем затвориться дверь, как я увидел в своем кабинет

In [119]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 20, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text3) - SEQLEN, STEP):
    input_chars.append(text3[i: i + SEQLEN])
    label_chars.append(text3[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 256, 256
NUM_ITERATIONS = 25 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


Итерация #: 0
1316/1316 [==============================] - 111s 85ms/step - loss: 2.4562
Генерация из посева: вя в городе, беспрес
вя в городе, беспрестоль о на моня в домо на моня в домо на моня в домо на моня в домо на моня в домо на моня в домо на ==================================================
Итерация #: 1
1316/1316 [==============================] - 114s 87ms/step - loss: 2.0865
Генерация из посева: ости. края раны не р
ости. края раны не разами от не стало от не стало от не стало от не стало от не стало от не стало от не стало от не стал==================================================
Итерация #: 2
1316/1316 [==============================] - 116s 88ms/step - loss: 1.9270
Генерация из посева: очался подстреленный
очался подстреленный веромино полодить с нем и проступил свою женщиный веромино полодить с нем и проступил свою женщиный==================================================
Итерация #: 3
1316/1316 [==============================] - 118s 90ms/step - loss: 1.8144
Генер

Для начала я очистил текст Алисы от не относящихся к рассказу текстов (в начале, в конце + некоторые слова и символы)

Увеличение итераций позволяет на более поздних итерациях получать все более осмысленный текст, что логично

Также небольшой эффект в повышение осмысленности добавило увеличение длины последовательности с 10 до 20. Осмысленный текст начал проглядываться немного раньше.

Я попробовал поменять текст на Повесть Чехова Драма на охоте, чтобы иметь возможность оценить всю полноту результата. Увеличил кол-во итераций до 25, BATCH_SIZE и HIDDEN_SIZE до 256. 

__В итоге, поскольку повесть была длиннее Алисы и кол-во итераций выше, то время на обучение сети выросло. Самый толковый результат из всех был получен на 20 итерации (набор фраз бессмысленный, но по крайней мере читабельный):__

__станется, — засудите, что он всего и смеясь, что он всего и следствие друг и не знал, что я не помню, как вы не понимаю,__

3. *Попробуйте на numpy реализовать нейронную сеть архитектуры LSTM

Без полноценного объяснения математики или самостоятельного изучения, сейчас я могу лишь тупо скопировать готовые примеры в сети. Не вижу в этом никакого смысла. Надеюсь, на уроке вы покажите и расскажите подробно про 3 и 4 ДЗ